# Part 1, PHPmyadmin

### Q1: Some of the facilities charge a fee to members, but some do not. Write a SQL query to produce a list of the names of the facilities that do.

SELECT name
From Facilities
WHERE membercost != 0; 

### Q2: How many facilities do not charge a fee to members?

SELECT count(name) as num_facilities
From Facilities
WHERE membercost != 0; 

### Q3: Write an SQL query to show a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost. Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

SELECT 
	facid, 
    name as "facility name", 
    membercost as "member cost", 
    monthlymaintenance as "monthly maintenance"
From Facilities
WHERE membercost < .20 * monthlymaintenance
	and membercost !=0

### Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5. Try writing the query without using the OR operator.

SELECT *
From Facilities
WHERE facid in(1,5)


### Q5: Produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than \$ 100. Return the name and monthly maintenance of the facilities in question.

SELECT 
	name,
    monthlymaintenance as "monthly maintenance cost",
    CASE
    	WHEN monthlymaintenance > 100 THEN 'expensive'
        ELSE 'cheap'
        END as "price label"
from Facilities


### Q6: You'd like to get the first and last name of the last member(s) who signed up. Try not to use the LIMIT clause for your solution.

SELECT 
	firstname as "first name",
    surname as "last name"
FROM
	Members
WHERE joindate = 
	(
        SELECT max(joindate)
        FROM Members
    )
    AND firstname != 'Guest'


### Q7: Produce a list of all members who have used a tennis court. Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name. 

SELECT
	Distinct f.name as "Facility",
    concat(firstname," ", surname) as "Member Name"
FROM Bookings as b
JOIN Facilities as f USING (facid)
JOIN Members as m USING (memid)
WHERE f.name LIKE 'Tennis%' 
	AND firstname != 'Guest'
ORDER BY "Member Name"


### Q8: Produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than \$30.Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.

SELECT
	Distinct f.name as "Facility",
    CASE 
    	WHEN firstname = 'Guest' Then 'Guest'
    	Else concat(firstname," ", surname)
    END as "Member Name",
    CASE
    	WHEN firstname = 'Guest' Then slots * guestcost *1.0
        ELSE slots * membercost * 1.0
    END as "Total Cost"
    
FROM Bookings as b
JOIN Facilities as f USING (facid)
JOIN Members as m USING (memid)
WHERE starttime LIKE '2012-09-14%'
AND (CASE
    	WHEN firstname = 'Guest' Then slots *guestcost
        ELSE slots * membercost
    END) > 30 
ORDER BY "Total Cost" desc

### Q9: This time, produce the same result as in Q8, but using a subquery.

SELECT * 
FROM (
    SELECT
    Distinct f.name as "Facility",
    CASE 
        WHEN firstname = 'Guest' Then 'Guest'
        Else concat(firstname," ", surname)
    END as "Member Name",
    CASE
        WHEN firstname = 'Guest' Then slots * guestcost * 1.0
        ELSE slots * membercost * 1.0
    END as "Total Cost"
    FROM Bookings as b
    JOIN Facilities as f USING (facid)
    JOIN Members as m USING (memid)
    WHERE starttime LIKE '2012-09-14%'
	)as subq
WHERE "Total Cost" > 30 
Order By "Total Cost"

# PART 2: SQLite Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook for the following questions.  



In [1]:
#imports
import sqlite3
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#create engine and connection
engine=create_engine('sqlite:///sqlite_db_pythonsqlite.db')
con = engine.connect()

In [3]:
#basic queries to view full tables
Bookings = con.execute('Select * From Bookings')
Facilities = con.execute('Select * From Facilities')
Members = con.execute('Select * From Members')

In [4]:
Bookings = pd.DataFrame(Bookings.fetchall())
Facilities = pd.DataFrame(Facilities.fetchall())
Members = pd.DataFrame(Members.fetchall())

###  Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [5]:
Q10 = pd.DataFrame(con.execute(
    'Select name, sum("costs") as "Total revenue ($)" \
    FROM ( \
        SELECT f.name, \
        firstname,  \
        membercost, \
        guestcost,  \
        slots,      \
        CASE        \
            WHEN firstname = "'"GUEST"'" THEN guestcost * slots \
            ELSE membercost * slots \
        END as "costs" \
        FROM Bookings as b \
        JOIN Facilities as f USING (facid)  \
        JOIN Members as m USING (memid) \
    ) \
    Group By name \
    HAVING sum("costs") <1000.0 \
    Order By "Total revenue ($)" desc'
).fetchall())

Q10

,name,Total revenue ($)
0,Pool Table,270
1,Snooker Table,240
2,Table Tennis,180


### Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [6]:
Q11 = pd.DataFrame(con.execute(
    'Select m1.firstname ||" "||m1.surname as "Member Name", m2.firstname || " " || m2.surname as "Recommended By" \
    From Members as m1 \
    Join Members as m2 \
        on m1.recommendedby = m2.memid \
    ORDER by m1.surname, m1.firstname'
    
).fetchall())

Q11

,Member Name,Recommended By
0,Florence Bader,Ponder Stibbons
1,Anne Baker,Ponder Stibbons
2,Timothy Baker,Jemima Farrell
3,Tim Boothe,Tim Rownam
4,Gerald Butters,Darren Smith
5,Joan Coplin,Timothy Baker
6,Erica Crumpet,Tracy Smith
7,Nancy Dare,Janice Joplette
8,Matthew Genting,Gerald Butters
9,John Hunt,Millicent Purview


### Q12: Find the facilities with their usage by member, but not guests

In [7]:
Q12 = pd.DataFrame(con.execute(
    'Select f.name as "Facility", firstname ||" "||surname as "Member Name", count(*) as " Usage" \
     FROM Bookings as b \
        JOIN Facilities as f USING (facid)  \
        JOIN Members as m USING (memid) \
    WHERE firstname != "GUEST"  \
    GROUP BY f.name, "Member Name"  \
    ORDER BY "Member Name"'
    
).fetchall())

Q12

,Facility,Member Name,Usage
0,Badminton Court,Anna Mackenzie,30
1,Massage Room 1,Anna Mackenzie,1
2,Pool Table,Anna Mackenzie,70
3,Snooker Table,Anna Mackenzie,7
4,Squash Court,Anna Mackenzie,2
...,...,...,...
197,Snooker Table,Tracy Smith,45
198,Squash Court,Tracy Smith,6
199,Table Tennis,Tracy Smith,28
200,Tennis Court 1,Tracy Smith,30


In [8]:
#Q13: Find the facilities usage by month, but not guests

Q13 = pd.DataFrame(con.execute(
    'Select f.name as "Facility", strftime("'"%m"'", starttime) as "Month", count(*) as " Usage" \
     FROM Bookings as b \
        JOIN Facilities as f USING (facid)  \
        JOIN Members as m USING (memid) \
    WHERE firstname != "GUEST"  \
    GROUP BY f.name, "Month"  \
    ORDER BY "name"'
    
).fetchall())

Q13



,Facility,Month,Usage
0,Badminton Court,07,51
1,Badminton Court,08,132
2,Badminton Court,09,161
3,Massage Room 1,07,77
4,Massage Room 1,08,153
5,Massage Room 1,09,191
6,Massage Room 2,07,4
7,Massage Room 2,08,9
8,Massage Room 2,09,14
9,Pool Table,07,103
